## Scraping banten.antarnews.com

In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [10]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [11]:
def scrape_links(page_number):
    url = f"https://banten.antaranews.com/seputar-banten/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [12]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 9 links from page 1
Total Links: 9


In [13]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "tags-wrapper"})
            if author_elem:
                author_text = author_elem.find('span')
                author_text = author_text.text.split(':')[1]
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix"})
            if body_elem:
                 content_text = body_elem.text
            else:
                content_text ="Content not found"
            nama_berita_match = re.search(r'https://(\w+\.\w+)\.com/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [14]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [15]:
df = pd.DataFrame(results)
print(len(results))
df.head(100)

9


,title,author,category,date,content,nama_berita,region,link
0,DLHK Tangerang lakukan uji emisi kendaraan ASN,Azmi Syamsul Ma'arif,Category not found,"Rabu, 27 September 2023 18:31 WIB",\n Tangerang (ANTARA) - Dinas Lin...,banten.antaranews,banten,https://banten.antaranews.com/berita/258735/dl...
1,Tradisi pengulasan golok setiap 12 Rabiul Awal...,Mulyana,Category not found,"Kamis, 28 September 2023 14:51 WIB",\n Serang (ANTARA) - Penjabat (Pj...,banten.antaranews,banten,https://banten.antaranews.com/berita/258846/tr...
2,Kawanan perampok gasak puluhan juta dari minim...,Azmi Syamsul Ma'arif,Category not found,"Rabu, 27 September 2023 14:30 WIB",\n Tangerang (ANTARA) - Kawanan p...,banten.antaranews,banten,https://banten.antaranews.com/berita/258720/ka...
3,Untirta-BCA kerjasama sistem monitoring pertum...,Lukman Hakim,Category not found,"Rabu, 27 September 2023 16:38 WIB","\nSerang, Banten (ANTARA) - Untirta melalui UP...",banten.antaranews,banten,https://banten.antaranews.com/berita/258717/un...
4,Wakil Bupati Serang Pandji Tirtayasa meninggal...,Desi Purnama Sari,Category not found,"Rabu, 27 September 2023 19:08 WIB",\n Serang (ANTARA) - Wakil Bupati...,banten.antaranews,banten,https://banten.antaranews.com/berita/258741/wa...
5,Pemilihan Ketua BPC PHRI Kota Cilegon segera d...,Mulyana,Category not found,"Rabu, 27 September 2023 20:06 WIB","\n Cilegon, Banten (ANTARA) - Pem...",banten.antaranews,banten,https://banten.antaranews.com/berita/258756/pe...
6,Warga Tangerang temukan potensi gas saat ngebo...,Azmi Syamsul Ma'arif,Category not found,"Rabu, 27 September 2023 16:37 WIB",\n Tangerang (ANTARA) - Warga Kam...,banten.antaranews,banten,https://banten.antaranews.com/berita/258714/wa...
7,Bupati Tatu sampaikan kabar duka dari Kabupate...,Lukman Hakim,Category not found,"Rabu, 27 September 2023 19:48 WIB","\n Serang, Banten (ANTARA) - Bupa...",banten.antaranews,banten,https://banten.antaranews.com/berita/258753/bu...
8,Ngarak Perahu Maulid diharapkan terus dilestar...,Achmad Irfan,Category not found,"Kamis, 28 September 2023 16:25 WIB",\n Tangerang (ANTARA) - Wali Kota...,banten.antaranews,banten,https://banten.antaranews.com/berita/258867/ng...


Tempat Simpan Save

In [16]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../../tempat_hasil_daerah/banten_antranews_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')